# Introduction and Purpose
The primary purpose of this notebook and set of methods/functions is to allow the DFCX script writer to copy/paste pages from one Flow to another Flow.  
In this notebook, we will demonstrate how to do this within the SAME agent, however these same methods/functions can be utilized to move pages BETWEEN agents!

## Imports
We're using the Legacy DFCX and DFFX libraries for this notebook.
Note that this scripting library is currently under a full refactor, so some of this code will slightly change in the future

In [1]:
# Copyright 2021 Google LLC. This software is provided as-is, without warranty
# or representation for any use or purpose. Your use of it is subject to your
# agreement with Google.

from dfcx_sapi.core.flow import Flow
from dfcx_sapi.dfcx.dfcx import DialogflowCX
from dfcx_sapi.dfcx.dfcx_functions import DialogflowFunctions

In [10]:
# Define your Service Account credentials file here, as well as the Agent ID you will be working on

ccai_creds = '/home/pmarlow/engineering/creds/verizon-custom-ccai-external-storage-key.json'
mfchat = 'projects/verizon-custom-ccai-external/locations/global/agents/28909604-2434-4173-86ec-4e47249e9c57'

In [11]:
# Instantiate your DFCX and DFFX Objects

dfcx = DialogflowCX(ccai_creds)
dffx = DialogflowFunctions(ccai_creds)

## Get Flows Map from Agent in Reverse so we can reference by Flow Display Name
The DFFX Class has a set of `map` functions that are handy for manipulating CX Resources when you only need the UUID and Display Name, but not the entire object.
Here, we need to reference our Flows by Display Name, so we can use the `get_flows_map` method, and set the `reverse` flag to `True` giving us the Display Names as the keys and the UUIDs as the values.

In [4]:
mfchat_flows_map = dffx.get_flows_map(mfchat, reverse=True)
mfchat_flows_map

{'Default Start Flow': 'projects/verizon-custom-ccai-external/locations/global/agents/28909604-2434-4173-86ec-4e47249e9c57/flows/00000000-0000-0000-0000-000000000000',
 'CMBLOCK': 'projects/verizon-custom-ccai-external/locations/global/agents/28909604-2434-4173-86ec-4e47249e9c57/flows/01c77870-df44-4582-9a04-343dcea8c3c8',
 'bill cycle ': 'projects/verizon-custom-ccai-external/locations/global/agents/28909604-2434-4173-86ec-4e47249e9c57/flows/1df77c2b-77a6-4e1e-8c51-55b67bc8c648',
 'PTP': 'projects/verizon-custom-ccai-external/locations/global/agents/28909604-2434-4173-86ec-4e47249e9c57/flows/39538e53-effe-4792-80cc-bb5710fe8552',
 'TRBL': 'projects/verizon-custom-ccai-external/locations/global/agents/28909604-2434-4173-86ec-4e47249e9c57/flows/44568c1c-646d-4e58-a4f1-0d4e97152071',
 'DEVP': 'projects/verizon-custom-ccai-external/locations/global/agents/28909604-2434-4173-86ec-4e47249e9c57/flows/481a86c4-347f-40c3-b4d6-df0d14898539',
 'TYS': 'projects/verizon-custom-ccai-external/locati

## Get All Pages from Source Flow
Once we have our Flows map, we can reference it in our call to extract the full list of pages from the source Flow in question.
In our case, we want to extract pages from a Flow named `AMNT15`.  
We will call the `list_pages` method from the DFCX Class and pass in the appropriate dict reference.

In [5]:
amnt15_pages = dfcx.list_pages(mfchat_flows_map['AMNT15'])
print('AMNT15 Page Count = {}'.format(len(amnt15_pages)))

AMNT15 Page Count = 76


## Extract Subset of Pages that we want to copy
Now that we have our List of Page objects, we need to extract the subset of pages that we plan on copying over.
Usually you will do this using some regex matcher or pattern to select your pages.  
The easiest way to do this is to ensure the the Page designer has prepended the `page.display_name` with a specific label.
The more unique the matching pattern, the better!

Ex:
- TYS - Page 1
- TYS - Page 2
- TYS - Page N

In [6]:
subset_pages = [] # define a list placeholder for your Page proto objects
for page in amnt15_pages:
    if 'TYS -' in page.display_name:
        subset_pages.append(page)

print('Total Pages to Copy = {}'.format(len(subset_pages)))

Total Pages to Copy = 13


## Create Page Shells in Destination Flow
Using the `subset_pages` that we just collected, iter through them and create a "shell" page in the destination flow.  
This allows CX to assign a new UUID for the destination page which we will use to replace all references in the existing `subset_pages`

Remember to pass in the `Destination Flow ID` using the reverse flows map from above.

**NOTE** - If you have a lot of pages, consider a `time.sleep(.5)` in your loop so as to not overrun your API limits!

In [8]:
for page in subset_pages:
    dfcx.create_page(mfchat_flows_map['TYS'], display_name=page.display_name) # remember to add kwarg for display_name

## Prep Source Page Objects
In a previous step, we collected the `subset_pages` list of Page objects.  
Now we will use a ~!* MAGIC *!~ `dffx` method called `convert_page_dependencies`

`convert_page_dependencies` has 2 modes:
- source
- destination

For `source` mode, you simply pass in the following:
- Agent ID String
- Original List of Page Objects (in our case, the `subset_pages` list) 
- 'source' keyword
- STRING DISPLAY NAME of the Source Flow

The `source` mode changes all of the UUIDs in the Page object to be PLAIN TEXT STRING DISPLAY NAMES using the internal map functions.  
You will want to collect your new MODIFIED page objects in a new variable, in our case here we call it `subset_pages_prepped`

For `destination` mode, you simply pass in the following:
- Agent ID String
- MODIFIED page objects that you received as output from the `source` mode pass of this same function
- 'destination' keyword
- STRING DISPLAY NAME of the Destination Flow

The `destination` mode will perform a reverse dictionary lookup on all of the previously modified resources using the internal map functions.
You will want to collect your new MODIFIED FINAL page objects in a new variable, in our case here we call it `final_pages`

In [7]:
subset_pages_prepped = dffx.convert_page_dependencies(mfchat, subset_pages, 'source', 'AMNT15')

In [8]:
final_pages = dffx.convert_page_dependencies(mfchat, subset_pages_prepped, 'destination', 'TYS')

## Update Pages in Destination Flow
Loop through your `final_pages` list and call the `update_page` function

In [9]:
for page in final_pages:
    dfcx.update_page(page.name, page)
    print('Updated Page: {}'.format(page.display_name))

Updated Page: TYS - Check DPP and monthly bill in credit
Updated Page: TYS - No pending orders
Updated Page: TYS - Submit TYS Request
Updated Page: TYS - Input form validation and save info
Updated Page: TYS - Check device eligibility
Updated Page: TYS - Show new Account Widget
Updated Page: TYS - Account role check
Updated Page: TYS - Show small business widget
Updated Page: TYS - Check pending orders
Updated Page: TYS - Get DeviceMDN
Updated Page: TYS - Show review widget
Updated Page: TYS - Follow up
Updated Page: TYS - Show Disclosure and Auth Widget


# Final Thoughts
All of the above could be improved by combining all of the functions into a single functions.
However, due to the shifting nature of bot building, I left the methods/functions separate to allow the script writer to make modifications or do sanity checks as they go along.

As noted at the start, this script could also be used to copy pages BETWEEN agents, and that is what it was originally written to do.